In [16]:
import requests,bs4,re,unicodedata
l2s=lambda s,l:s.join(map(str,l))
default_year="2024"

class Course:
    """
    講義情報を扱うクラス
    """
    def __init__(self):
        self.full_id="" #講義番号
        self.title="" #講義名
        self.classroomstr="" #教室
        self.semlist=[] #開講学期
        self.datelist=[] #曜日・時限
        self.description="" #内容
        self.infocols=["講義番号","講義名","教室","開講学期","曜日・時限"]
        self.infovals=[] #講義情報をまとめたリスト
        self.syllabus_url="" #シラバスURL
        self.moodle_url="" #moodleURL
    
    def get_info(self):
        """
        シラバスから情報を取得し、オブジェクトにセットします。
        """
        syllabus_url="https://kyomu.adm.okayama-u.ac.jp/Portal/Public/Syllabus/SyllabusSearchStart.aspx?lct_year="+default_year+"&lct_cd="+self.full_id+"&je_cd=1"
        moodle_url="https://moodle.el.okayama-u.ac.jp/course/view.php?idnumber="+self.full_id
        res=requests.get(syllabus_url,timeout=1)
        res.raise_for_status() #ウェブページを取得出来ているか確認
        soup=bs4.BeautifulSoup(res.text,"html.parser")
        elems=soup.select("span")
        #テキスト処理
        text=[]
        for elem in elems:
            word=elem.getText().split(',')
            text.append(word)
        #取得した情報をset
        self.title=text[11][0] 
        self.semlist=[int(unicodedata.normalize("NFKC",x)) for x in re.findall(r"\d",text[6][0])]
        self.datelist=text[30]
        self.classroomstr=','.join(text[22])
        self.syllabus_url=syllabus_url
        self.moodle_url=moodle_url
        self.description+="シラバスURL: "+syllabus_url+"\n"+"moodleURL: "+moodle_url
        self.infovals=[self.full_id,self.title,self.classroomstr,l2s(',',self.semlist)+"学期",l2s(',',self.datelist)]
    
    def show_info(self):
        """
        オブジェクトにセットされている情報を表示します。
        例：
        講義番号: 2024098456
        講義名: ＵＮＩＸプログラミング
        教室: 工学部１号館大講義室
        開講学期: 1学期
        曜日・時限: 月5〜6,木1〜2
        """
        infostr=""
        for col,val in zip(self.infocols,self.infovals):
            infostr+=col+": "+val+"\n"
        print(infostr)

In [19]:
from google.oauth2 import service_account
from googleapiclient.errors import HttpError
from googleapiclient.discovery import build
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv("../env/.env")

#strとdatetimeを相互に変換する関数の定義
strptime=lambda str,format:datetime.datetime.strptime(str,format)
strftime=lambda dt,format:datetime.datetime.strftime(dt,format)
date2weekday={0:"月",1:"火",2:"水",3:"木",4:"金",5:"土",6:"日"}
weekday2date={'月':0,'火':1,'水':2,'木':3,'金':4,'土':5,'日':6}

#google calendar_apiのための認証情報
creds = service_account.Credentials.from_service_account_file('../env/credentials.json')
service = build('calendar', 'v3', credentials=creds)
calendar_id = os.getenv('MAIL_ADDRESS')  #自分のメールアドレス

class Schedule(Course):
    """
    グーグルカレンダーの予定を扱うクラスです。
    講義情報を扱うクラス(Course)の派生クラスです。
    """
    def __init__(self):
        super().__init__()
        #時限と時刻の対応
        self.period2time={
        1:["08:40","09:30"],
        2:["09:40","10:30"],
        3:["10:45","11:35"],
        4:["11:45","12:35"],
        5:["13:25","14:15"],
        6:["14:25","15:15"],
        7:["15:30","16:20"],
        8:["16:30","17:20"]
        }
        self.sem2date=self.get_sem() #学期と日付の対応
        self.starttime_dict={} #講義の開始時刻
        self.endtime_dict={} #講義の終了時刻
    def get_sem(self):
        """
        semester.csvから学期と日付の対応(開始日・終了日)を取得します。
        """
        df = pd.read_csv('../config/semester.csv',index_col="学期")
        self.sem2date=dict(zip(list(df.index),df.values.tolist()))
        return self.sem2date
    def get_time(self):
        """
        講義情報から学期・曜日ごとの予定作成時刻を求めます。
        """
        for sem in self.semlist:
            self.starttime_dict[sem]={}
            self.endtime_dict[sem]={}
            date_sem=[strptime(sem2date[sem][tf], "%Y/%m/%d") for tf in [0,1]] #第n学期の日付の始終
            for date in self.datelist:
                self.starttime_dict[sem][date]=[]
                self.endtime_dict[sem][date]=[]
                weekday=date[0] #開講曜日
                period=[int(i) for i in re.findall(r"\d+", date)] #開講時限の始終
                time=[strptime(period2time[period[tf]][tf],"%H:%M") for tf in [0,1]] #開講時間の始終
                today=datetime.datetime.now() #今日の日付
                #今日の日付を基準に、開講する日付までの曜日のずれを数え、その分日付を進める。(曜日を揃える)
                delta_day=datetime.timedelta(days=(weekday2date[weekday]-today.weekday())%7)
                hour=[time[0].hour,time[1].hour]
                minute=[time[0].minute,time[1].minute]
                dt=lambda tf:datetime.datetime(today.year,today.month,today.day,hour[tf],minute[tf])+delta_day
                #曜日を揃えた日付をstartとする。
                start=dt(0)
                end=dt(1)
                while True:
                    #start < 開講学期の初日 => start + 7day
                    if(start<=date_sem[0]):
                        start+=datetime.timedelta(days=7)
                        end+=datetime.timedelta(days=7)
                        continue
                    #開講学期の初日 < start < 開講学期の最終日 => 時刻を格納し、start + 7day
                    elif((date_sem[0]<=start)&(start<=date_sem[1]+datetime.timedelta(days=1))):
                        self.starttime_dict[sem][date].append(start)
                        self.endtime_dict[sem][date].append(end)
                        start+=datetime.timedelta(days=7)
                        end+=datetime.timedelta(days=7)
                        continue 
                    #それ以外ならループを抜ける
                    else:
                        break
    def create_event(self,start_time,end_time):
        """
        Googleカレンダーに予定を作成します。
        """
        event = {
            'summary': self.title,
            'location':self.classroomlist,
            'description':self.description,
            'start': {
                'dateTime': strftime(start_time,'%Y-%m-%dT%H:%M:%S'),
                'timeZone': 'Asia/Tokyo',
            },
            'end': {
                'dateTime': strftime(end_time,'%Y-%m-%dT%H:%M:%S'),
                'timeZone': 'Asia/Tokyo',
            },
        }
        try:
            event = service.events().insert(calendarId=calendar_id, body=event).execute()
            print(f'Event created: {event.get("htmlLink")}')
        except HttpError as error:
            print(f'An error occurred: {error}')
            event = None

course=Schedule()
course.full_id="2024098456"
course.get_info()
course.show_info()

講義番号: 2024098456
講義名: ＵＮＩＸプログラミング
教室: 工学部１号館大講義室
開講学期: 1学期
曜日・時限: 月5〜6,木1〜2

{1: ['2024/4/8', '2024/6/6'], 2: ['2024/6/7', '2024/8/5'], 3: ['2024/10/1', '2024/11/28'], 4: ['2024/11/29', '2024/2/7']}


In [22]:
# サンプルコード
import tkinter as tk
from tkinter import ttk
import webbrowser

class Application(tk.Frame):
    """
    ウインドウのクラスです。
    """
    def __init__(self, master=None):
        # Windowの初期設定を行う。
        super().__init__(master)
        # Windowの画面サイズを設定する。
        # geometryについて : https://kuroro.blog/python/rozH3S2CYE0a0nB3s2QL/
        self.master.geometry("300x200")
        
        #講義情報のLabelframe
        l_frame = ttk.Labelframe(
            root,
            relief="ridge",
            text="講義情報",  #タイトルの設定
            labelanchor="n",    #タイトル位置の設定
            )
        #label1
        for col,val in zip(course.infocols,course.infovals):
            l = tk.Label(l_frame,text=col+": "+val)
            #ラベルを左寄せ
            l.pack(anchor=tk.W)
        #label2
        syllabus_url=tk.Label(l_frame,text="シラバス",fg="blue")
        syllabus_url.pack()
        syllabus_url.bind("<Button-1>",lambda e:self.link_click(course.syllabus_url))
        #label3
        moodle_url=tk.Label(l_frame,text="moodle",fg="blue")
        moodle_url.pack()
        moodle_url.bind("<Button-1>",lambda e:self.link_click(course.moodle_url))
        
        l_frame.pack(padx=3,pady=3)
    
    def link_click(self,url):
        """
        ハイパーリンクをウェブブラウザで開きます。
        """
        webbrowser.open_new(url)

if __name__ == "__main__":
    root = tk.Tk()
    app = Application(master=root)
    app.mainloop()